# Proyecto BI - Análitica de Texto según ODS

## Etapa 1

## Objetivos

*   Aplicar la metodología de analítica de textos para la construcción de soluciones de analítica alineadas con los objetivos del negocio en un contexto de aplicación.
*   Planear la interacción con un grupo interdisciplinario para identificar usuarios y posibles herramientas a desarrollar para la interacción del resultado del modelo desarrollado.

## Problema
<p style="text-align: justify;"> Desarrollar un modelo de clasificación, con técnicas de aprendizaje automático, que permita relacionar de manera automática un texto según los ODS. Al igual que desarrollar una aplicación que facilite la interacción con el resultado de dicho modelo. El modelo podrá ser utilizado entonces para la interpretación y análisis de la información textual que es recopilada a través de diferentes fuentes por UNFPA en procesos de planeación participativa para el desarrollo a nivel territorial.


### 1. Instalación e importanción de librerías.

In [29]:
!pip install ftfy

In [30]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [31]:
!pip install unidecode

In [32]:
# Librería para manejar las contracciones que se presentan en el inglés.
!pip install contractions

In [33]:
# librería para manejar las flexiones gramaticales en el idioma inglés.
!pip install inflect
!pip install pandas-profiling==2.7.1

ERROR: Could not find a version that satisfies the requirement pandas-profiling==2.7.1 (from versions: 3.0.0, 3.1.0, 3.2.0, 3.3.0, 3.4.0, 3.5.0, 3.6.0, 3.6.1, 3.6.2, 3.6.3, 3.6.4, 3.6.5, 3.6.6)
ERROR: No matching distribution found for pandas-profiling==2.7.1


In [34]:
# librería Natural Language Toolkit, usada para trabajar con textos 
import nltk
# Punkt permite separar un texto en frases.
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paula\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [35]:
# Descarga de paquete WordNetLemmatizer, este es usado para encontrar el lema de cada palabra
# ¿Qué es el lema de una palabra? ¿Qué tan dificil puede ser obtenerlo, piensa en el caso en que tuvieras que escribir la función que realiza esta tarea?
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\paula\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
# Instalación de librerias
import pandas as pd
import numpy as np
import sys
from pandas_profiling import ProfileReport
from unidecode import unidecode

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import nltk
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
nltk.download('wordnet')


from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
#from sklearn.metrics import plot_precision_recall_curve
from sklearn.base import BaseEstimator, ClassifierMixin
from nltk.tokenize import word_tokenize
from sklearn.metrics import (
    ConfusionMatrixDisplay, RocCurveDisplay,
    roc_auc_score, precision_score, recall_score, f1_score
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt


In [37]:
import warnings
warnings.filterwarnings("ignore")

In [38]:
# Configuring pandas to show all cell content
pd.set_option("display.max_colwidth", None)

In [39]:
# Downloading stopwords
nltk.download("punkt")
nltk.download("stopwords")
stop_words = stopwords.words("spanish")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paula\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
stop_words


['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

### 2. Perfilamiento y entendimiento de los datos


#### Lectura de los datos.

In [41]:
# Uso de la libreria pandas para la lectura de archivos excel
df = pd.read_excel('./data/cat_345.xlsx')
# Asignación a una nueva variable de los datos leidos
data_t=df


In [42]:
data_t.shape

(3000, 2)

In [43]:
data_t.sample(5)

,Textos_espanol,sdg
872,"Las puntuaciones de la calidad de vida relacionada con la salud fueron significativamente más bajas para los encuestados con EDA en comparación con otros encuestados. En consecuencia, la pérdida de productividad por enfermedad de larga duración y discapacidad aumentó en las personas que habían sufrido ADE en comparación con otros encuestados. Si los pacientes están en edad de trabajar, el empeoramiento de la salud causado por los daños puede impedirles participar en la vida laboral, lo que se traduce en una pérdida de años de vida productiva. Más allá del impacto negativo sobre la autoestima y las pérdidas económicas personales derivadas de la reducción de ingresos, los efectos derivados incluyen mayores necesidades de apoyo y cuidados, mayor dependencia de la asistencia social y pérdida de ingresos fiscales.",3
1130,"Este último tema es particularmente importante cuando el material curricular o las tareas educativas tienen poca relevancia o significado para los grupos dentro de la población estudiantil, incluidos los estudiantes indígenas (Young, 2010). En comparación con otros estudiantes en sus propios países, los tres grupos de bajo rendimiento realizan menos tareas cada semana, y este indicador es particularmente pronunciado entre los alumnos de bajo rendimiento en Australia (Figura 6.3).",4
2165,"Cuando empezó, se fijó un objetivo: ""Demostrar a los concejales varones y a los demás que podía hacerlo"". Durante el estudio de Gender Links, At the Coalface: El género en el gobierno local, afirmó que tiene más confianza en sí misma gracias al apoyo que le ha prestado el Grupo Multipartidista de Mujeres (MWC) y a un proyecto único de capacitación en el que participó, el proyecto piloto Género, Gobierno Local y Comunicaciones, emprendido con Gender Links y la Universidad de Witwatersrand de octubre de 2004 a mayo de 2005. También he aprendido más sobre la formulación y aplicación de políticas y los procesos que intervienen en su elaboración.",5
1496,"Con el tiempo, considere alejarse del financiamiento compartido y aumentar los subsidios de cupones lo suficiente para compensar esto. Si es necesario, los asistentes de maestros y maestros de educación especial capacitados brindan apoyo adicional bajo la supervisión del maestro. Cerca del 30% de la población estudiantil recibe este apoyo cada año (Field, Kuczera y Pont, 2007). Si todas estas medidas fallan, un equipo multidisciplinario integrado por maestros, trabajadores sociales, psicólogos y representantes de las autoridades de salud y vivienda pública, si es necesario, elabora un plan de apoyo.",4
214,"Los países más avanzados en su transición demográfica (Chile y Cuba) presentan tasas de mortalidad geriátrica superiores a la media regional. De hecho, una comparación entre las situaciones de Haití y Cuba muestra que Haití tardará hasta 2050 en registrar las tasas de mortalidad en la población mayor de 65 años que Cuba había registrado seis décadas antes. Cuba tiene una estructura de edad más avanzada que Haití y, por lo tanto, su población tiene más probabilidades de morir por encima de los 65 años. En el gráfico IV.2, que muestra la proporción de muertes por enfermedades crónicas y degenerativas en función de la edad media de la población (2002), se observa en un extremo a los países más avanzados en la transición demográfica (Argentina, Barbados, Chile, Cuba y Uruguay), con más del 80% de las muertes por esas causas y, en el otro, a los países a punto de entrar en plena transición demográfica con un proceso de envejecimiento aún incipiente (Guatemala, Haití, Honduras y Nicaragua, entre otros). Esto puede ilustrarse comparando la esperanza de vida al nacer y la esperanza de vida ajustada por salud (EVAS) al nacer.",3


## 2.1 Limpieza de datos

In [44]:
data_t["sdg"].value_counts(dropna=False, normalize=True)

sdg
3    0.333333
4    0.333333
5    0.333333
Name: proportion, dtype: float64

Se puede ver una distribución equitativa de los datos en los tres grupos de ODS

In [45]:
data_t["Textos_espanol"] = data_t["Textos_espanol"].astype(str)

### Corrección de palabras mal codificadas

In [46]:
import ftfy

def fix_malformed_words(text):
    # Utiliza ftfy para corregir problemas de codificación
    text = ftfy.fix_text(text)
    return text

# Aplica la función a tu DataFrame
data_t['Textos_espanol'] = data_t['Textos_espanol'].apply(fix_malformed_words)



In [47]:
#Se verifica que se haya realizado la corrección
data_t.head(5)

,Textos_espanol,sdg
0,"Por ejemplo, el número de consultas externas de especialistas es de 319 por cada mil derechohabientes en el SP, en comparación con 338 y 620 por cada mil derechohabientes en el IMSS y el ISSSTE, respectivamente. Si bien algunas de estas diferencias pueden reflejar una necesidad desigual (como la población ligeramente mayor del ISSSTE), otras no pueden justificarse de esta manera. El número de recetas que no pudieron ser surtidas en su totalidad por un farmacéutico debido a la falta de existencias es de 33% dentro del SP en comparación con 14% dentro del IMSS según los datos de la encuesta (aunque las propias cifras de los institutos de la SS sugieren tasas más altas de recetas surtidas). Ambas cifras se encuentran entre las más altas de la OCDE. El gasto de bolsillo no se ha reducido significativamente en la última década, a pesar de los esfuerzos para lograr la cobertura sanitaria universal a través de la reforma del SP.",3
1,"En 2007, el gobierno central financió directamente solo el 0,3% del gasto total en salud, pero realizó transferencias específicas para el gasto en salud que ascendieron a otro 5,6% del gasto total. La mayor parte del gasto presupuestario en salud lo realizan los gobiernos de los condados. Por lo tanto, los condados pobres solo pueden ofrecer un bajo nivel de atención a la población local. El gobierno provincial proporciona subsidios del lado de la oferta a los hospitales.",3
2,"Claramente, hay muchos otros factores en juego, en particular, una variedad de barreras y determinantes sociales. Estos pueden estar relacionados con el género, la clase, la etnia, la casta, el idioma y la religión, y surgen de estructuras, normas y procesos sociales arraigados que aceptan e incluso fomentan distribuciones injustas de la riqueza y los recursos sociales. Por ejemplo, las personas que viven con el VIH y algunas otras enfermedades crónicas enfrentan altos niveles de estigma y discriminación que dificultan su acceso a pruebas médicas, tratamiento, atención y apoyo. Por lo tanto, los gobiernos que buscan mejorar los estándares de salud querrán mejorar la condición de la mujer, brindar servicios de salud reproductiva efectivos, implementar programas integrales de lactancia materna en línea con las recomendaciones actuales y aumentar las presiones comunitarias contra el aborto selectivo por sexo. Las personas pobres, que carecen de educación e información, o de fondos para opciones más saludables, son más vulnerables al impacto de las ENT que los ricos y es probable que mueran antes. Directa e indirectamente, las ENT tendrán impactos de gran alcance en el progreso hacia los ODM.",3
3,"Por ejemplo, el estado australiano de Victoria creó una agencia de promoción de la salud financiada con los impuestos sobre el tabaco, y varios estados de EE.UU. (California, Massachusetts, Arizona y Oregón) han destinado parte de los ingresos procedentes de los impuestos especiales sobre el tabaco a actividades de educación y control del tabaco, mientras que en Nueva Jersey, Ohio y Texas los ingresos procedentes del alcohol se utilizan para programas de educación y prevención relacionados con el alcohol (Sassi et al., 2004), Los impuestos locales pueden ser una fuente importante de ingresos, sobre todo en los sistemas sanitarios descentralizados (por ejemplo, Dinamarca, Finlandia, Italia y Suecia)17 . Además, el nivel de los impuestos locales puede fijarse en función de las preferencias locales sobre el nivel y la calidad de los servicios médicos. Sin embargo, en ausencia de un mecanismo de transferencia ajustado al riesgo y gestionado centralmente, la descentralización reduce la mancomunación de riesgos y podría dar lugar a mayores disparidades sanitarias interregionales.",3
4,"El consumo anual de alcohol se estima en 15,7 litros al año para los hombres, más de un 50% por encima de la media de la OCDE (10,4) (Sistema mundial de información sobre alcohol y salud de la OMS), lo que puede ayudar a explicar la alta mor

### Eliminación de caracteres especiales y puntuación

In [48]:
import re
import unicodedata
import inflect
from nltk.corpus import stopwords

# Esta lista contiene las stop words en español
spanish_stopwords = set(stopwords.words('spanish'))

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_words.append(word.lower())
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all integer occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in spanish_stopwords:
            new_words.append(word)
    return new_words

def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words


### Tokenización

In [49]:
data_t['Textos_espanol'] = data_t['Textos_espanol'].apply(contractions.fix) #Aplica la corrección de las contracciones

In [50]:
data_t['words'] = data_t['Textos_espanol'].apply(word_tokenize).apply(preprocessing) #Aplica la eliminación del ruido
data_t.head()

,Textos_espanol,sdg,words
0,"Por ejemplo, el número de consultas externas de especialistas es de 319 por cada mil derechohabientes en el SP, en comparación con 338 y 620 por cada mil derechohabientes en el IMSS y el ISSSTE, respectivamente. Si bien algunas de estas diferencias pueden reflejar una necesidad desigual (como la población ligeramente mayor del ISSSTE), otras no pueden justificarse de esta manera. El número de recetas que no pudieron ser surtidas en su totalidad por un farmacéutico debido a la falta de existencias es de 33% dentro del SP en comparación con 14% dentro del IMSS según los datos de la encuesta (aunque las propias cifras de los institutos de la SS sugieren tasas más altas de recetas surtidas). Ambas cifras se encuentran entre las más altas de la OCDE. El gasto de bolsillo no se ha reducido significativamente en la última década, a pesar de los esfuerzos para lograr la cobertura sanitaria universal a través de la reforma del SP.",3,"[ejemplo, numero, consultas, externas, especialistas, three hundred and nineteen, cada, mil, derechohabientes, sp, comparacion, three hundred and thirtyeight, six hundred and twenty, cada, mil, derechohabientes, imss, issste, respectivamente, si, bien, diferencias, pueden, reflejar, necesidad, desigual, poblacion, ligeramente, mayor, issste, pueden, justificarse, manera, numero, recetas, pudieron, ser, surtidas, totalidad, farmaceutico, debido, falta, existencias, thirtythree, dentro, sp, comparacion, fourteen, dentro, imss, segun, datos, encuesta, aunque, propias, cifras, institutos, ss, sugieren, tasas, mas, altas, recetas, surtidas, ambas, cifras, encuentran, mas, altas, ocde, gasto, bolsillo, reducido, significativamente, ultima, decada, pesar, esfuerzos, lograr, cobertura, sanitaria, universal, traves, reforma, sp]"
1,"En 2007, el gobierno central financió directamente solo el 0,3% del gasto total en salud, pero realizó transferencias específicas para el gasto en salud que ascendieron a otro 5,6% del gasto total. La mayor parte del gasto presupuestario en salud lo realizan los gobiernos de los condados. Por lo tanto, los condados pobres solo pueden ofrecer un bajo nivel de atención a la población local. El gobierno provincial proporciona subsidios del lado de la oferta a los hospitales.",3,"[two thousand and seven, gobierno, central, financio, directamente, solo, 03, gasto, total, salud, realizo, transferencias, especificas, gasto, salud, ascendieron, 56, gasto, total, mayor, parte, gasto, presupuestario, salud, realizan, gobiernos, condados, condados, pobres, solo, pueden, ofrecer, bajo, nivel, atencion, poblacion, local, gobierno, provincial, proporciona, subsidios, lado, oferta, hospitales]"
2,"Claramente, hay muchos otros factores en juego, en particular, una variedad de barreras y determinantes sociales. Estos pueden estar relacionados con el género, la clase, la etnia, la casta, el idioma y la religión, y surgen de estructuras, normas y procesos sociales arraigados que aceptan e incluso fomentan distribuciones injustas de la riqueza y los recursos sociales. Por ejemplo, las personas que viven con el VIH y algunas otras enfermedades crónicas enfrentan altos niveles de estigma y discriminación que dificultan su acceso a pruebas médicas, tratamiento, atención y apoyo. Por lo tanto, los gobiernos que buscan mejorar los estándares de salud querrán mejorar la condición de la mujer, brindar servicios de salud reproductiva efectivos, implementar programas integrales de lactancia materna en línea con las recomendaciones actuales y aumentar las presiones comunitarias contra el aborto selectivo por sexo. Las personas pobres, que carecen de educación e información, o de fondos para opciones más saludables, son más vulnerables al impacto de las ENT que los ricos y es probable que mueran antes. Directa e indirectamente, las ENT tendrán impactos de gran alcance en el progreso hacia los ODM.",3,"[claramente, factores, juego, particular, variedad, barreras, determinantes, sociales, 

### Normalización de texto

In [52]:
# Tokenización

# Lematización
def lemmatize_words(words):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words]

# Stemming (eliminación de prefijos y sufijos)
def stem_words(words):
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_words(words)
    return stems + lemmas

# Aplicar tokenización y lematización
data_t['words'] = data_t['Textos_espanol'].apply(tokenize)
data_t['words'] = data_t['words'].apply(lemmatize_words)

# Aplicar tokenización y eliminación de prefijos y sufijos (stemming)
data_t['words'] = data_t['Textos_espanol'].apply(tokenize)
data_t['words'] = data_t['words'].apply(stem_words)

data_t.head()


NameError: name 'get_wordnet_pos' is not defined

In [ ]:
#Se verifica que se haya realizado la corrección
data_t.head(5)

,Textos_espanol,sdg
0,por ejemplo el número de consultas externas de especialistas es de 319 por cada mil derechohabientes en el sp en comparación con 338 y 620 por cada mil derechohabientes en el imss y el issste respectivamente si bien algunas de estas diferencias pueden reflejar una necesidad desigual como la población ligeramente mayor del issste otras no pueden justificarse de esta manera el número de recetas que no pudieron ser surtidas en su totalidad por un farmacéutico debido a la falta de existencias es de 33 dentro del sp en comparación con 14 dentro del imss según los datos de la encuesta aunque las propias cifras de los institutos de la ss sugieren tasas más altas de recetas surtidas ambas cifras se encuentran entre las más altas de la ocde el gasto de bolsillo no se ha reducido significativamente en la última década a pesar de los esfuerzos para lograr la cobertura sanitaria universal a través de la reforma del sp,3
1,en 2007 el gobierno central financió directamente solo el 03 del gasto total en salud pero realizó transferencias específicas para el gasto en salud que ascendieron a otro 56 del gasto total la mayor parte del gasto presupuestario en salud lo realizan los gobiernos de los condados por lo tanto los condados pobres solo pueden ofrecer un bajo nivel de atención a la población local el gobierno provincial proporciona subsidios del lado de la oferta a los hospitales,3
2,claramente hay muchos otros factores en juego en particular una variedad de barreras y determinantes sociales estos pueden estar relacionados con el género la clase la etnia la casta el idioma y la religión y surgen de estructuras normas y procesos sociales arraigados que aceptan e incluso fomentan distribuciones injustas de la riqueza y los recursos sociales por ejemplo las personas que viven con el vih y algunas otras enfermedades crónicas enfrentan altos niveles de estigma y discriminación que dificultan su acceso a pruebas médicas tratamiento atención y apoyo por lo tanto los gobiernos que buscan mejorar los estándares de salud querrán mejorar la condición de la mujer brindar servicios de salud reproductiva efectivos implementar programas integrales de lactancia materna en línea con las recomendaciones actuales y aumentar las presiones comunitarias contra el aborto selectivo por sexo las personas pobres que carecen de educación e información o de fondos para opciones más saludables son más vulnerables al impacto de las ent que los ricos y es probable que mueran antes directa e indirectamente las ent tendrán impactos de gran alcance en el progreso hacia los odm,3
3,por ejemplo el estado australiano de victoria creó una agencia de promoción de la salud financiada con los impuestos sobre el tabaco y varios estados de eeuu california massachusetts arizona y oregón han destinado parte de los ingresos procedentes de los impuestos especiales sobre el tabaco a actividades de educación y control del tabaco mientras que en nueva jersey ohio y texas los ingresos procedentes del alcohol se utilizan para programas de educación y prevención relacionados con el alcohol sassi et al 2004 los impuestos locales pueden ser una fuente importante de ingresos sobre todo en los sistemas sanitarios descentralizados por ejemplo dinamarca finlandia italia y suecia17 además el nivel de los impuestos locales puede fijarse en función de las preferencias locales sobre el nivel y la calidad de los servicios médicos sin embargo en ausencia de un mecanismo de transferencia ajustado al riesgo y gestionado centralmente la descentralización reduce la mancomunación de riesgos y podría dar lugar a mayores disparidades sanitarias interregionales,3
4,el consumo anual de alcohol se estima en 157 litros al año para los hombres más de un 50 por encima de la media de la ocde 104 sistema mundial de información sobre alcohol y salud de la oms lo que puede ayudar a explicar la alta mortalidad por enfermedad hepática relacionada con el alcohol si bien la prevalencia de la obesidad e